In [27]:
# Libraries required for Neural Network
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import time 
import copy
import torch
import torch.optim as optim
from torch.autograd import grad
from torch.autograd import Variable
import torch.nn as nn
dtype = torch.float

%matplotlib inline

In [28]:
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

0
1
NVIDIA GeForce MX230
True


In [29]:
device = 'cpu'
# For plots
plt.rcParams.update({'font.size':16})

### Define general functions

In [30]:
# Define the sin() activation function
class mySin(torch.nn.Module):
    @staticmethod
    def activation_function(input):
        return torch.sin(input)

In [31]:
# Calculate the derivative with auto-differention
def df_dx(x,f):
    pts = torch.ones(x.shape, dtype=dtype, device=torch.device(device))
    return grad([f],[x],grad_outputs=pts,create_graph=True)[0]

In [32]:
# Stochastic perturbation of the evaluation points
# force t[0]=t0 & force points to be in the t-interval
def perturbation(grid,t0,tf,sig=0.5):
    delta_t = grid[1] - grid[0]
    noise = delta_t * torch.randn_like(grid) * sig
    t = grid + noise
    t.data[2] = torch.ones(1,1)*(-1)
    t.data[t<t0] = t0 - t.data[t<t0]
    t.data[t>tf] = 2*tf - t.data[t>tf]
    t.data[0] = torch.ones(1,1)*t0
    t.data[-1] = torch.ones(1,1)*tf
    t.requires_grad = False
    return t

In [33]:
# Trial or parametric solution for boundary contitions 
def parametric_solution(t,nn,t0,x1,endpoint):
    N1,N2 = nn(t)
    # There are two parametric solutions, we take some that satisfy dirichlet conditions of radial part of hydrogen atom
    f = 1-torch.exp(t-endpoint)
    psi_hat = f*N1
    return psi_hat

In [34]:
# Radial differential equation or eigenvalue problem of hamiltonian operator
def hamiltonian_equation(t,psi,E,V):
    dpsi_dx = df_dx(t,psi)
    dpsi_dxx = df_dx(t,dpsi_dx)
    f = dpsi_dxx + dpsi_dx*2/t + (2*E+V)*psi
    L = (f.pow(2)).mean()
    var_loss = (f.pow(2)).var()    # variance of f
    H_psi = -1*dpsi_dxx/2 + V*psi
    return L, f, H_psi, var_loss 

In [35]:
# Gives the potential at each point r
def potential(Xs):
    l = 0
    Xsnp = Xs.data.numpy()     # points from tensor to numpy
    Vnp = 2/Xsnp - l*(l+1)/Xsnp**2     # potential over numpy arrays
    Vtorch = torch.from_numpy(Vnp) 
    return Vtorch

### Define Neural Network model

In [36]:
class qNN(torch.nn.Module):
    def __init__(self, n_hl=10):      # neurons in hidden layer
        super(qNN,self).__init__()

        # Define the activation function
        self.activ_func = mySin()
        # Define layers
        self.E_in = torch.nn.Linear(1,1)
        self.Lin_1 = torch.nn.Linear(2,n_hl)
        self.Lin_2 = torch.nn.Linear(n_hl,n_hl)
        self.out = torch.nn.Linear(n_hl,1)

    def forward(self,t):
            
        In_1 = self.E_in(torch.ones_like(t))
        L1 = self.Lin_1(torch.cat((t,In_1),1))
        h1 = self.activ_func(L1)
        L2 = self.Lin_2(h1)
        h2 = self.activ_func(L2)
        out = self.out(h2)

        return out, -1*torch.abs(In_1)   # Negative energy because eigenvalues are negatives for hydrogen atom

In [37]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

### Training of Neural Network model

In [38]:
def training(t0, tf, x1, neurons, epochs, n_train, lr, minibatch_number=1):
    '''        
        Inputs
        t0 -- left point on domain
        tf -- right point on domain
        x1 --
        neurons -- number of units at hidden layer
        epochs -- number of iterations of training
        n_train -- number of points between t0 & tf including them
        lr -- learning rate 
        minibatch_number -- size of minibatch for points on domain

        Outputs 

    '''
    
    par2 = 0
    model_0 = qNN(neurons)
    model_0 = model_0.to(device)
    model_1 = 0
    betas = [0.999,0.9999]
    optimizer = optim.Adam(model_0.parameters(), lr=lr, betas=betas)

    # List's to save variables
    Loss_history = []         ;         Llim = 1e+20    # Upper limit of loss
    En_loss_history = []
    boundary_loss_history = []
    nontriv_loss_history = []
    SE_loss_history = []
    Ennontriv_loss_history = []
    criteria_loss_history = []
    En_history = []
    prob_loss = []
    EWall_history = []
    orth_losses = []
    var_loss_history = []
    fc_ground = 0
    fc_first_excited = 0
    reinit = 0
    orthtime = 2e+4

    di = (None, 1e+20)
    dic = {}
    for i in range(1000):
        dic[i] = di

    grid = torch.linspace(t0,tf,n_train).rechape(-1,1)

    # Training iterations
    TeP0 = time.time()
    walle = 0

    for tt in range(epochs):
        
        t = torch.abs(perturbation(grid,t0,tf,sig=0.03*tf)) + 1e-1

        # Batching
        batch_size = int(n_train/minibatch_number)
        batch_start, batch_end = 0, batch_size

        idx = np.random.permutation(n_train)
        t_b = t[idx]
        t_b.requires_grad = True
        t_f = t[-1]
        t_f = t_f.reshape(-1,1)
        t_f.requires_grad = True
        loss = 0
        
        # Reinitialize weights at orthtime
        if tt == orthtime:
            model_0.apply(weights_init)

        for nbatch in range(minibatch_number):
            # Batch time set
            t_mb = t_b[batch_start:batch_end].to(device)
            norm_loss_reg = 1.0

            # Neural Network solutions
            nn, En = model_0(t_mb)
            En_history.append(En[0].data.tolist()[0])  # Append energy values from tensor to scalar (not list)

            psi = parametric_solution(t_mb, nn, )